In [1]:
from KKT_NN import KKT_NN
from pickle import load
import numpy as np
import torch
import random
import pandas as pd
from pandas.plotting import parallel_coordinates
import seaborn as sns

In [ ]:
random.seed(42)     # python random generator
np.random.seed(42)  # numpy random generator
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

In [2]:
torch.nn.Softplus(beta=5)(torch.tensor([-1.0]))

tensor([0.0013])

In [3]:
model = KKT_NN()
model.kinn.load_state_dict(torch.load("kinn.pt", map_location=torch.device("cpu")))

<All keys matched successfully>

In [4]:
sobol = torch.quasirandom.SobolEngine(7, scramble=True, seed=42)

In [5]:
batch = sobol.draw(10000)

actions =2*batch[..., :2] -1
P_pots = 2*batch[...,2] -1
P_max = 2*batch[...,3] -1
Q_max = 2*batch[...,4] -1
P_plus = 2*batch[...,5] -1
Q_plus = 2*batch[...,6] -1

In [6]:
stationarity, feasibility, complementary = model.kkt_loss(actions, P_pots, P_max, Q_max, P_plus, Q_plus,)

In [7]:
df = pd.DataFrame(torch.stack([actions[..., 0], actions[...,1],P_pots, P_max, Q_max, P_plus, Q_plus, stationarity, feasibility, complementary, stationarity+feasibility+complementary], 1).cpu().detach(), columns=['action_p', 'action_q', 'P_pots', 'P_max', 'Q_max', 'P_plus', 'Q_plus', 'stat', 'feas', 'compl', 'sum'])
df.to_csv("param_analysis.csv", index = False)

In [8]:
df

,action_p,action_q,P_pots,P_max,Q_max,P_plus,Q_plus,stat,feas,compl,sum
0,0.995027,-0.791267,0.645958,-0.161136,0.056644,-0.887519,-0.089415,1.824964,0.004653,0.000171,1.829787
1,-0.792331,0.963542,-0.360671,0.721019,-0.023555,0.318015,0.285633,0.799294,0.000000,0.001765,0.801059
2,-0.036059,-0.482156,0.144591,-0.828831,-0.562205,0.609388,-0.861613,0.740230,0.000000,0.000053,0.740283
3,0.239611,0.278630,-0.859534,0.273357,0.528979,-0.181120,0.513404,1.180743,0.000000,0.002635,1.183378
4,0.415527,-0.122091,-0.042011,0.181854,0.879252,-0.739524,-0.738558,0.787828,0.000000,0.000919,0.788747
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.266101,0.927778,0.068175,0.967921,0.656921,-0.904058,-0.685016,0.278728,0.010647,0.001352,0.290728
9996,0.074582,-0.709347,0.780163,0.612288,0.806007,-0.455030,0.566916,0.021438,0.000000,0.000266,0.021704
9997,-0.154235,0.537798,-0.244893,-0.052405,-0.776642,0.773954,-0.808070,0.661008,0.000000,0.000760,0.661768
9998,-0.892927,-0.017229,0.280859,0.382094,-0.309493,0.044073,0.356692,0.272252,0.000000,0.000441,0.272693


In [9]:
df.corr()

,action_p,action_q,P_pots,P_max,Q_max,P_plus,Q_plus,stat,feas,compl,sum
action_p,1.000000,-0.000157,-0.000266,-0.000576,-0.000113,-0.001194,0.000070,0.663741,0.022480,0.153203,0.663901
action_q,-0.000157,1.000000,-0.000028,0.000316,-0.000161,-0.000004,-0.001501,-0.005678,-0.006763,0.279811,-0.005475
P_pots,-0.000266,-0.000028,1.000000,-0.000001,0.000596,-0.000008,0.000167,-0.254003,0.123706,-0.588026,-0.254064
P_max,-0.000576,0.000316,-0.000001,1.000000,0.000023,-0.000021,-0.000205,-0.148571,0.038951,0.450217,-0.148076
Q_max,-0.000113,-0.000161,0.000596,0.000023,1.000000,-0.000062,0.000158,-0.230558,0.068399,0.124281,-0.230226
P_plus,-0.001194,-0.000004,-0.000008,-0.000021,-0.000062,1.000000,-0.000005,0.013799,-0.110096,-0.014420,0.013433
Q_plus,0.000070,-0.001501,0.000167,-0.000205,0.000158,-0.000005,1.000000,-0.008560,-0.093334,0.069737,-0.008803
stat,0.663741,-0.005678,-0.254003,-0.148571,-0.230558,0.013799,-0.008560,1.000000,-0.026832,0.167137,0.999995
feas,0.022480,-0.006763,0.123706,0.038951,0.068399,-0.110096,-0.093334,-0.026832,1.000000,-0.056163,-0.023664
compl,0.153203,0.279811,-0.588026,0.450217,0.124281,-0.014420,0.069737,0.167137,-0.056163,1.000000,0.167751
